In [1]:
import tensorflow as tf


In [2]:
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.environments import gym_wrapper
import tf_agents
from tf_agents.environments import utils
from tf_agents.environments import wrappers
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common
from tf_agents.utils import composite
from tf_agents.environments import parallel_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.metrics import py_metrics
from tf_agents.policies import random_tf_policy
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import policy_step
from tf_agents.trajectories import time_step as ts
from tf_agents.trajectories import trajectory
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils


In [3]:
from functions import m_get_data,splitdf
import numpy as np
import pandas as pd
import datetime
import gym
import numpy as np
import plotly.graph_objects as go
import random

from collections import deque
import MetaTrader5 as mt5

In [4]:
def get_data(currency,):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2015, 1, 1), datetime.datetime.now()))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data

In [5]:
currency_pair = 'GOLD'
selected_timeframes = ['D1', 'H4', 'H1','M15','M5','M1']
start_date = '2015-01-01'
end_date = datetime.datetime.now()

ohlc_data_dict = m_get_data(currency_pair, selected_timeframes, start_date, end_date)


In [6]:

dat=get_data('GOLD')
dfs=splitdf(dat,2)
dataf=dfs[0]


c:\Users\DELL\anaconda3\envs\agent00\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
df = dataf.iloc[:1000]


In [14]:
eval_data = dataf.iloc[1000:2000]


In [8]:
def candle_info(open,close,high=0,low=0):
    candle_size = 0
    candle_spread = 0
    candle_type = {}

    if open > close:
        candle_size=open-close
        candle_spread=high-low
        candle_type={'candletype':0,'bodysize':candle_spread,'size':candle_size}
        
    elif open < close:
        candle_size=close-open
        candle_spread=high-low
        candle_type={'candletype':1,'bodysize':candle_spread,'size':candle_size}
    return candle_type

In [9]:
df['candle_info'] = df.apply(lambda row: candle_info(row['Open'], row['Close'], row['High'], row['Low']), axis=1)

# Extracting new columns from the dictionary in the 'candle_info' column
df = pd.concat([df, pd.json_normalize(df['candle_info'])], axis=1)

# Dropping the original 'candle_info' column
df = df.drop(columns=['candle_info'])
df


C:\Users\DELL\AppData\Local\Temp\ipykernel_14868\1672786796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['candle_info'] = df.apply(lambda row: candle_info(row['Open'], row['Close'], row['High'], row['Low']), axis=1)


,time,Open,High,Low,Close,tick_volume,spread,real_volume,candletype,bodysize,size
0,1420185600,1186.07,1187.03,1186.07,1187.03,97,50,0,1.0,0.96,0.96
1,1420186500,1187.03,1187.03,1186.53,1186.83,62,51,0,0.0,0.50,0.20
2,1420187400,1186.83,1187.23,1186.03,1186.34,145,40,0,0.0,1.20,0.49
3,1420188300,1186.35,1186.99,1186.35,1186.79,89,54,0,1.0,0.64,0.44
4,1420189200,1186.94,1187.04,1185.43,1185.53,183,43,0,0.0,1.61,1.41
...,...,...,...,...,...,...,...,...,...,...,...
995,1421631900,1277.63,1278.53,1276.69,1278.53,304,44,0,1.0,1.84,0.90
996,1421632800,1278.63,1278.93,1277.59,1278.48,541,40,0,0.0,1.34,0.15
997,1421633700,1278.37,1278.93,1277.73,1278.82,362,39,0,1.0,1.20,0.45
998,1421634600,1278.77,1279.02,1277.22,1277.52,479,40,0,0.0,1.80,1.25


In [18]:
eval_data['candle_info'] = eval_data.apply(lambda row: candle_info(row['Open'], row['Close'], row['High'], row['Low']), axis=1)

# Extracting new columns from the dictionary in the 'candle_info' column
eval_data = pd.concat([eval_data, pd.json_normalize(eval_data['candle_info'])], axis=1)

# Dropping the original 'candle_info' column
eval_data = eval_data.drop(columns=['candle_info'])
eval_data


C:\Users\DELL\AppData\Local\Temp\ipykernel_14868\1865562065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data['candle_info'] = eval_data.apply(lambda row: candle_info(row['Open'], row['Close'], row['High'], row['Low']), axis=1)


,time,Open,High,Low,Close,tick_volume,spread,real_volume,candletype,bodysize,size
1000,1.421636e+09,1278.27,1278.61,1276.31,1277.40,1027.0,40.0,0.0,NaN,NaN,NaN
1001,1.421637e+09,1277.30,1278.29,1275.96,1276.41,794.0,41.0,0.0,NaN,NaN,NaN
1002,1.421638e+09,1276.41,1276.64,1275.33,1276.44,582.0,40.0,0.0,NaN,NaN,NaN
1003,1.421639e+09,1276.49,1276.89,1275.32,1275.44,440.0,40.0,0.0,NaN,NaN,NaN
1004,1.421640e+09,1275.39,1277.53,1274.89,1277.23,590.0,39.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.63,1.21
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.35,0.25
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.10,0.19
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.40,0.05


In [10]:
from tf_agents.specs import tensor_spec

In [11]:
class simple(gym.Env):
    def __init__(self, initial_balance, data):
        super(simple, self).__init__()

        # Define the OHLC data
        ohlc_data = data[['time', 'Open', 'High', 'Low', 'Close', 'tick_volume', 'candletype', 'bodysize', 'size']].values
        max_price = max(ohlc_data[:, 2])
        using_high = (int(max_price)) + 15
        self.ohlc_data = ohlc_data
        self.current_index = 0
        self.current_trade = None
        self.current_balance = initial_balance
        self.total_profit = 0.0

        # Define the observation space (using close prices only for simplicity)
        self.observation_space = gym.spaces.Box(low=0, high=using_high, shape=(1,))
        self.action_space = gym.spaces.Discrete(2)
        

    def count_profit(self, tradetype, entry, price):
        pr = price - entry
        if tradetype == 0:
            return {'profit': max(pr, 0), 'loss': max(-pr, 0)}
        elif tradetype == 1:
            return {'profit': max(-pr, 0), 'loss': max(pr, 0)}

    def _process_trade(self, profit):
        self.current_balance += profit['profit'] - profit['loss']
        self.total_profit += profit['profit'] - profit['loss']

    def step(self, action):
        reward = 0
        if self.current_trade is None:
            if action == 0:  # Buy
                self.current_trade = {'action': 0, 'entry_price': self.ohlc_data[self.current_index][4],
                                      'index': self.ohlc_data[self.current_index][0]}
            elif action == 1:  # Sell
                self.current_trade = {'action': 1, 'entry_price': self.ohlc_data[self.current_index][4],
                                      'index': self.ohlc_data[self.current_index][0]}

        if self.current_trade is not None:
            if action == 0:
                reward = 0
            elif action == 1:
                self.current_trade['exit_price'] = self.ohlc_data[self.current_index][4]
                current_price = self.ohlc_data[self.current_index][4]
                entry_price = self.current_trade['entry_price']
                trade_type = self.current_trade['action']
                index = self.current_trade['index']
                info = self.ohlc_data[self.ohlc_data[:,0] == index]
                entry_price = info[0,4]
                profit = self.count_profit(trade_type, entry_price, current_price)
                profit_value = profit['profit'] - profit['loss']
                reward = profit_value

                self._process_trade(profit)

        done = None
        if self.current_index >= len(self.ohlc_data) - 1:
          done=True
        


        state = np.array([self.ohlc_data[self.current_index][4]])  # Assuming single value for observation
        discount = 0.99  # Set your desired discount factor
        step_type = 0  # Set the appropriate value for step_type

        # Construct the time step as a dictionary
        time_step = {'step_type': step_type, 'reward': reward, 'discount': discount, 'observation': state}
        info = {}
        if not done:
          self.current_index += 1
        return state, reward, done, info

    def render(self, mode='human'):
        if mode == 'human':
            print(f"Total Cumulative Profit: {self.total_profit}")
        else:
            raise NotImplementedError("Unsupported rendering mode")

    def reset(self):
        self.current_index = 0
        self.current_trade = None
        self.total_profit = 0.0
        self.current_balance = 0
        return np.array([self.ohlc_data[self.current_index][4]])


In [16]:
envn =simple(100,df)
train = gym_wrapper.GymWrapper(envn)
train_env = tf_py_environment.TFPyEnvironment(train)


In [19]:
eval_envn =simple(100,eval_data)
eval = gym_wrapper.GymWrapper(eval_envn)
eval_env = tf_py_environment.TFPyEnvironment(eval)



In [20]:
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)


In [21]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)

train_step_counter = tf.Variable(0)

agent00 = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent00.initialize()

In [22]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),train_env.action_spec())

In [24]:
def compute_avg_return(environment, policy, num_episodes):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]



In [31]:
compute_avg_return(train_env,random_policy,6)

5715.3257

In [33]:
from tf_agents.replay_buffers import py_uniform_replay_buffer

In [34]:
replay_buffer_capacity = 1000*32 # same capacity as the TFUniformReplayBuffer

py_replay_buffer = py_uniform_replay_buffer.PyUniformReplayBuffer(
    capacity=replay_buffer_capacity,
    data_spec=tensor_spec.to_nest_array_spec(agent00.collect_data_spec))

In [35]:
from tf_agents.metrics import py_metrics
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_episode_driver

In [36]:
data_generated=300
driver = dynamic_episode_driver.DynamicEpisodeDriver(
    train_env, random_policy, [py_replay_buffer.add_batch], num_episodes=data_generated).run()


In [37]:
dataset = py_replay_buffer.as_dataset(
    sample_batch_size=170,
    num_steps=2)

iterator = iter(dataset)

In [38]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots



In [39]:
import matplotlib.pyplot as plt

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
step = 5: Average Return = 0.0
step = 10: Average Return = -6440.16943359375
step = 15: loss = 1554.1385498046875
step = 15: Average Return = -6440.16943359375
step = 20: Average Return = -6440.16943359375
step = 25: Average Return = -6440.16943359375
step = 30: loss = 851.2975463867188
step = 30: Average Return = -6440.16943359375
step = 35: Average Return = -6440.16943359375
step = 40: Average Return = 0.0
step = 45: loss = 818.8226318359375
step = 45: Average Return = -6440.16943359375
step = 50: Average Return = 0.0
step = 55: Average Return = -6440.16943359375
step = 60: loss = 949.3755493164062
step = 60: Average Return = -6440.16943359375
step = 65: Average Return = -6440.16943359375
step = 70: Average Return = -6440.16943359375
step = 75: loss = 26

In [42]:
agent00.train = common.function(agent00.train)
agent00.train_step_counter.assign(0)
Btrain_eval=6
avg_return = compute_avg_return(eval_env, agent00.policy,Btrain_eval)
returns = [avg_return]
time_step = train_env.reset()
num_train_steps = 200
eval_interval=5
log_interval=15

for _ in range(num_train_steps):
  
  trajectories= next(iterator)
  loss = agent00.train(experience=trajectories).loss
  step = agent00.train_step_counter.numpy()
  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step,loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent00.policy, Btrain_eval)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)
  print(f"Step: {_}, Loss: {loss}")
  
steps = range(0, num_train_steps + 1, eval_interval)
plt.plot(steps, returns, label='Average Return', marker='o')
plt.xlabel('Training Steps')
plt.ylabel('Average Return')
plt.title('Learning Curve')
plt.legend()
plt.show()


Step: 0, Loss: 1589.5621337890625
Step: 1, Loss: 1238.207763671875
Step: 2, Loss: 875.03076171875
Step: 3, Loss: 945.2112426757812
step = 5: Average Return = 0.0
Step: 4, Loss: 1390.146240234375
Step: 5, Loss: 1593.919677734375
Step: 6, Loss: 1042.485107421875
Step: 7, Loss: 979.4314575195312
Step: 8, Loss: 1116.19970703125
step = 10: Average Return = -6440.16943359375
Step: 9, Loss: 1320.8382568359375
Step: 10, Loss: 1474.2666015625
Step: 11, Loss: 1205.7783203125
Step: 12, Loss: 1145.71435546875
Step: 13, Loss: 873.74365234375
step = 15: loss = 190462.6875
step = 15: Average Return = -6440.16943359375
Step: 14, Loss: 190462.6875
Step: 15, Loss: 191564.71875
Step: 16, Loss: 1967.47119140625
Step: 17, Loss: 2783.0078125
Step: 18, Loss: 3084.1796875
step = 20: Average Return = -6440.16943359375
Step: 19, Loss: 1960.4840087890625
Step: 20, Loss: 1060.2701416015625
Step: 21, Loss: 995.3258056640625
Step: 22, Loss: 419842.4375
Step: 23, Loss: 1879.286376953125


KeyboardInterrupt: 

In [91]:
num_train_steps = 10
losses = []

# Create a figure with subplots outside the loop
fig = make_subplots(rows=1, cols=1, subplot_titles=["Learning Curve"])

for _ in range(num_train_steps):
    # Perform training step and get loss
    trajectories = next(iterator)
    loss = agent00.train(experience=trajectories)
    
    # Log the loss
    losses.append(loss)

    # Update the trace within the loop
    trace_loss = go.Scatter(x=list(range(len(losses))), y=losses, mode='lines', name='Loss')
    fig.update_traces(trace_loss)

# Update layout
fig.update_layout(title_text='DQN Training Learning Curve', xaxis_title='Step', yaxis_title='Loss')

# Show the figure after the loop
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [88]:


num_train_steps = 10

for _ in range(num_train_steps):
  
  trajectories= next(iterator)
  loss = agent00.train(experience=trajectories)
  print(f"Step: {_}, Loss: {loss}")

Step: 0, Loss: LossInfo(loss=<tf.Tensor: shape=(), dtype=float32, numpy=4444.081>, extra=DqnLossInfo(td_loss=<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([3.71375084e+01, 2.18851791e+02, 5.67285156e+00, 3.18721867e+00,
       4.75345612e-01, 2.30758423e+02, 2.16074203e+02, 2.14235321e+02,
       8.47491980e-01, 2.14610703e+02, 2.70490150e+01, 2.32527130e+02,
       2.16074203e+02, 8.29996185e+01, 1.55944973e-02, 1.73852203e+02,
       9.83173084e+00, 2.16074203e+02, 2.13976334e+02, 2.42897854e+01,
       2.23857040e+02, 2.56807281e+02, 2.23857040e+02, 5.31074047e+00,
       2.47309532e+01, 1.89818561e+00, 3.70861969e+01, 2.12576904e+02,
       2.59667328e+02, 1.83453064e+01, 2.01411606e+02, 3.58127441e+01,
       8.54360819e-01, 1.83453064e+01, 7.23839111e+01, 5.38974991e+01,
       2.23857040e+02, 2.78965851e+02, 2.04868622e+02, 1.73517593e+02,
       3.03729877e+01, 2.06289825e+02, 2.27931854e+02, 1.00588020e+02,
       2.17917343e+02, 2.49446777e+02, 2.32527130e+02, 2.08873

In [80]:

# Initial driver.run will reset the environment and initialize the policy.
final_time_step, policy_state = driver.run()

print('final_time_step', final_time_step)


final_time_step TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1187.03]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>})


In [55]:
for _ in range(10):
    time_step = train_env.reset()

    while not time_step.is_last():
        action_step = agent00.policy.action(time_step)

        action = action_step.action.numpy()

        next_time_step = train_env.step(action)
        experience = {
            key: getattr(time_step, key) for key in agent00.collect_data_spec._fields
        }
        replay_buffer.add_batch(experience)
        experience = replay_buffer.gather_all()

        # Calculate the loss and train the agent
        loss_info = agent00.train(experience)
        

        # Update the time step
        time_step = next_time_step

    # Update the training step counter
train_step_counter.assign_add(1)

    # Optionally, log/print metrics or other information
print(f'Episode: {num_episodes.result().numpy()}, Total Steps: {env_steps.result().numpy()}')

    # Reset metrics for the next episode
num_episodes.reset()
env_steps.reset()


AttributeError: 'TimeStep' object has no attribute 'action'

In [12]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=4,
    num_steps=2)

iterator = iter(dataset)

num_train_steps = 30

for _ in range(num_train_steps):
  trajectories, _ = next(iterator)
  loss = agent00.train(experience=trajectories)

NameError: name 'replay_buffer' is not defined

In [40]:
replay_buffer_capacity = 10000
batch_size = 64  # Set your desired batch size here
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent00.collect_data_spec,
    batch_size=batch_size,
    max_length=replay_buffer_capacity
)
